# Functionality of this notebook: 

* calculate the ciCOH for normal and mild LFP data in rest

## Import Packages

In [1]:
import os, sys
import scipy.io as sio
import glob
import numpy as np
import matplotlib.pyplot as plt
import pickle 
import math
import pandas as pd
%matplotlib inline

In [2]:
# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)

# import_nbmodule used for import package in .ipynb
import import_nbmodule

# import util/folder_extract.pynb 
from util.folder_extract import exp_subfolders, code_corresfolder

# import ciCoherence_overtime in connAnalyTool/synchronization_indices.ipynb
from connAnalyTool.synchronization_indices import ciCoherence_acrosstrials
from connAnalyTool.synchronization_indices import ciCoherence_overtime

importing Jupyter notebook from /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/util/folder_extract.ipynb
importing Jupyter notebook from /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/connAnalyTool/synchronization_indices.ipynb


## exp subfolders & code_corresfolder

In [6]:
_, _, pipelinefolder, _= exp_subfolders()

In [7]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [8]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder, correparentfolder = code_corresfolder(os.getcwd(), nb_name)

## input setup

In [9]:
inputfolder = os.path.join(pipelinefolder, 'NHP_Pinky', '0_dataPrep', 'Rest', 'm5_restData_segNarrowedDownsample')

In [10]:
variablesinLoadfile = ['lfpsegs_m1', 'lfpsegs_GM', 'lfpsegs_stn', 'lfpsegs_gp', 'fs', 
                   'chans_m1', 'GMChnAreas', 'segsRemain']

In [11]:
chnInf_folder = correparentfolder
chnInf_file = os.path.join(chnInf_folder, 'chn_brainArea_simCoord_M1ThaSMADBS.csv')

## global parameters

In [12]:
freq = [26, 28]

## Save Setup

In [13]:
savefolder = corresfolder
savefilename =  'ciCOH_rest' + '_freq' + str(freq[0]) + '_' + str(freq[1]) + '.pickle'

## extract lfp

In [14]:
def lfpallfiles_extract(files):
    if 'lfpdata' in locals():
        del lfpdata
    
    for i, file in enumerate(files):
        
        ### load data
        matdat = sio.loadmat(file, variable_names = variablesinLoadfile, 
                             struct_as_record = False, squeeze_me = True) 

        # fs: sample rate
        fs = matdat['fs'] 
        
        
        ### extract the noused channels, only calculate once
        if i == 0:
            GMChnAreas = matdat['GMChnAreas'].tolist()
            chni_notused = []
            for chni, area in enumerate(GMChnAreas):
                if len(area) == 0:
                    # this channel is not used for any area
                    chni_notused.append(chni)
                    
            # remove the used channels from GMChnAreas and chans_GM
            chans_GM = [*range(101, 133,1)]
            for idx in sorted(chni_notused, reverse=True):
                del GMChnAreas[idx]
                del chans_GM[idx]
                
            # chnInf_recchni
            recchni_used = matdat['chans_m1'].tolist() + chans_GM + [*range(0, 7,1)] + [*range(0, 7,1)]
        
        
        

        ### dealing lfp data
        
        # lfp (np.ndarray): ntemporal * nchns
        lfp_m1,  lfp_GM = matdat['lfpsegs_m1'],  matdat['lfpsegs_GM']
        lfp_stn, lfp_gp = matdat['lfpsegs_stn'], matdat['lfpsegs_gp']
        
        
        # remove the lfp_GM data of the channel not used
        lfp_GM = np.delete(lfp_GM, chni_notused, axis = 1) 
        

        # combine the data of all the channels
        lfpdata_1file = np.concatenate((lfp_m1, lfp_GM, lfp_stn, lfp_gp), axis = 1)
        
        # concatenate to lfpdata for all files
        if 'lfpdata' not in locals():
            lfpdata = lfpdata_1file
        else:
            lfpdata = np.concatenate((lfpdata, lfpdata_1file), axis = 2)
                
                
    
    return lfpdata, recchni_used

In [15]:
files_normal = glob.glob(os.path.join(inputfolder, '*normal*'))
files_mild = glob.glob(os.path.join(inputfolder, '*mild*'))
files_moderate = glob.glob(os.path.join(inputfolder, '*moderate*'))

In [16]:
lfpwins_normal, recchni_used = lfpallfiles_extract(files_normal)
lfpwins_mild, _ = lfpallfiles_extract(files_mild)
lfpwins_moderate, _ = lfpallfiles_extract(files_moderate)

## Extract chnInf using recchni_used

In [17]:
### load channel information ###
df = pd.read_csv(chnInf_file, header = 0)

In [18]:
# recording chni 
recording_chni = df['recording_chni'].tolist()

chnInf_brainarea = df['brainarea']
chnInf_matchni = df['mat_chni']

# coord
x, y = df['simulated_x'].to_numpy(), df['simulated_y'].to_numpy()
chnInf_coord = np.concatenate((np.expand_dims(x, axis = 1), np.expand_dims(y, axis = 1)), axis = 1)

In [19]:
### extract the idx_used in list recording_chni
idx_used = []
# get the idx_used for m1 and GM (i.e. except stn and gp) in list recording_chni
for i, rechni in enumerate(recchni_used[0:-14]):
    idx_used.append(recording_chni.index(rechni))
    
# get idx of stn and gp (i.e. the last 14 channels)
idx_used = idx_used + [*range(len(recording_chni)-14, len(recording_chni))]

In [20]:
# get chnInf for used channels
chnInf_brainarea = chnInf_brainarea[idx_used].tolist()
chnInf_coord = chnInf_coord[idx_used,:]
chnInf_matchni = chnInf_matchni[idx_used]

chnInf_recchni = recchni_used

## Calculate ciCOH

### balance mild and normal trials

In [21]:
# select the smaller trial number
ntrials_normal, ntrials_mild,  ntrials_moderate = lfpwins_normal.shape[2], lfpwins_mild.shape[2], lfpwins_moderate.shape[2]
ntrials = min([ntrials_normal, ntrials_mild,  ntrials_moderate])

# balance trials
lfpwins_normal = lfpwins_normal[:,:,0:ntrials]
lfpwins_mild = lfpwins_mild[:,:,0:ntrials]
lfpwins_moderate = lfpwins_moderate[:,:,0:ntrials]

###  normal ciCOH

In [22]:
lfpwins_allfiles = lfpwins_normal

### calculate ciCOH
ntempo, nchns, nwins = lfpwins_allfiles.shape
ciCOH_allWins = np.zeros((nchns, nchns, nwins))
for wini in range(nwins):
    print("wini = " + str(wini) + "/" + str(nwins))
    for chni in range(nchns -1):
        signal1 = lfpwins_allfiles[:, chni, wini]
        
        for chnj in range(chni+1, nchns):
            signal2 = lfpwins_allfiles[:, chnj, wini]
            
            # ciCOH_allWins assignment
            ciCOH_allWins[chni, chnj, wini] = ciCoherence_overtime(signal1, signal2)
            
            # symmetrical
            ciCOH_allWins[chnj, chni, wini] = ciCOH_allWins[chni, chnj, wini]
            
            del signal2
        del signal1

wini = 0/343
wini = 1/343
wini = 2/343
wini = 3/343
wini = 4/343
wini = 5/343
wini = 6/343
wini = 7/343
wini = 8/343
wini = 9/343
wini = 10/343
wini = 11/343
wini = 12/343
wini = 13/343
wini = 14/343
wini = 15/343
wini = 16/343
wini = 17/343
wini = 18/343
wini = 19/343
wini = 20/343
wini = 21/343
wini = 22/343
wini = 23/343
wini = 24/343
wini = 25/343
wini = 26/343
wini = 27/343
wini = 28/343
wini = 29/343
wini = 30/343
wini = 31/343
wini = 32/343
wini = 33/343
wini = 34/343
wini = 35/343
wini = 36/343
wini = 37/343
wini = 38/343
wini = 39/343
wini = 40/343
wini = 41/343
wini = 42/343
wini = 43/343
wini = 44/343
wini = 45/343
wini = 46/343
wini = 47/343
wini = 48/343
wini = 49/343
wini = 50/343
wini = 51/343
wini = 52/343
wini = 53/343
wini = 54/343
wini = 55/343
wini = 56/343
wini = 57/343
wini = 58/343
wini = 59/343
wini = 60/343
wini = 61/343
wini = 62/343
wini = 63/343
wini = 64/343
wini = 65/343
wini = 66/343
wini = 67/343
wini = 68/343
wini = 69/343
wini = 70/343
wini = 71/343
wi

In [23]:
ciCOH = np.mean(ciCOH_allWins, axis = 2)
ciCOH_normal = ciCOH
del ciCOH, ciCOH_allWins

### mild ciCOH

In [24]:
lfpwins_allfiles = lfpwins_mild

### calculate ciCOH
ntempo, nchns, nwins = lfpwins_allfiles.shape
ciCOH_allWins = np.zeros((nchns, nchns, nwins))
for wini in range(nwins):
    print("wini = " + str(wini) + "/" + str(nwins))
    for chni in range(nchns -1):
        signal1 = lfpwins_allfiles[:, chni, wini]
        
        for chnj in range(chni+1, nchns):
            signal2 = lfpwins_allfiles[:, chnj, wini]
            
            # ciCOH_allWins assignment
            ciCOH_allWins[chni, chnj, wini] = ciCoherence_overtime(signal1, signal2)
            
            # symmetrical
            ciCOH_allWins[chnj, chni, wini] = ciCOH_allWins[chni, chnj, wini]
            
            del signal2
        del signal1

wini = 0/343
wini = 1/343
wini = 2/343
wini = 3/343
wini = 4/343
wini = 5/343
wini = 6/343
wini = 7/343
wini = 8/343
wini = 9/343
wini = 10/343
wini = 11/343
wini = 12/343
wini = 13/343
wini = 14/343
wini = 15/343
wini = 16/343
wini = 17/343
wini = 18/343
wini = 19/343
wini = 20/343
wini = 21/343
wini = 22/343
wini = 23/343
wini = 24/343
wini = 25/343
wini = 26/343
wini = 27/343
wini = 28/343
wini = 29/343
wini = 30/343
wini = 31/343
wini = 32/343
wini = 33/343
wini = 34/343
wini = 35/343
wini = 36/343
wini = 37/343
wini = 38/343
wini = 39/343
wini = 40/343
wini = 41/343
wini = 42/343
wini = 43/343
wini = 44/343
wini = 45/343
wini = 46/343
wini = 47/343
wini = 48/343
wini = 49/343
wini = 50/343
wini = 51/343
wini = 52/343
wini = 53/343
wini = 54/343
wini = 55/343
wini = 56/343
wini = 57/343
wini = 58/343
wini = 59/343
wini = 60/343
wini = 61/343
wini = 62/343
wini = 63/343
wini = 64/343
wini = 65/343
wini = 66/343
wini = 67/343
wini = 68/343
wini = 69/343
wini = 70/343
wini = 71/343
wi

In [25]:
ciCOH = np.mean(ciCOH_allWins, axis = 2)
ciCOH_mild = ciCOH
del ciCOH, ciCOH_allWins

### Moderate ciCOH

In [26]:
lfpwins_allfiles = lfpwins_moderate

### calculate ciCOH
ntempo, nchns, nwins = lfpwins_allfiles.shape
ciCOH_allWins = np.zeros((nchns, nchns, nwins))
for wini in range(nwins):
    print("wini = " + str(wini) + "/" + str(nwins))
    for chni in range(nchns -1):
        signal1 = lfpwins_allfiles[:, chni, wini]
        
        for chnj in range(chni+1, nchns):
            signal2 = lfpwins_allfiles[:, chnj, wini]
            
            # ciCOH_allWins assignment
            ciCOH_allWins[chni, chnj, wini] = ciCoherence_overtime(signal1, signal2)
            
            # symmetrical
            ciCOH_allWins[chnj, chni, wini] = ciCOH_allWins[chni, chnj, wini]
            
            del signal2
        del signal1

wini = 0/343
wini = 1/343
wini = 2/343
wini = 3/343
wini = 4/343
wini = 5/343
wini = 6/343
wini = 7/343
wini = 8/343
wini = 9/343
wini = 10/343
wini = 11/343
wini = 12/343
wini = 13/343
wini = 14/343
wini = 15/343
wini = 16/343
wini = 17/343
wini = 18/343
wini = 19/343
wini = 20/343
wini = 21/343
wini = 22/343
wini = 23/343
wini = 24/343
wini = 25/343
wini = 26/343
wini = 27/343
wini = 28/343
wini = 29/343
wini = 30/343
wini = 31/343
wini = 32/343
wini = 33/343
wini = 34/343
wini = 35/343
wini = 36/343
wini = 37/343
wini = 38/343
wini = 39/343
wini = 40/343
wini = 41/343
wini = 42/343
wini = 43/343
wini = 44/343
wini = 45/343
wini = 46/343
wini = 47/343
wini = 48/343
wini = 49/343
wini = 50/343
wini = 51/343
wini = 52/343
wini = 53/343
wini = 54/343
wini = 55/343
wini = 56/343
wini = 57/343
wini = 58/343
wini = 59/343
wini = 60/343
wini = 61/343
wini = 62/343
wini = 63/343
wini = 64/343
wini = 65/343
wini = 66/343
wini = 67/343
wini = 68/343
wini = 69/343
wini = 70/343
wini = 71/343
wi

In [27]:
ciCOH = np.mean(ciCOH_allWins, axis = 2)
ciCOH_moderate = ciCOH
del ciCOH, ciCOH_allWins

## save ciCOH

In [28]:
ciCOH = dict()
ciCOH['normal'], ciCOH['mild'], ciCOH['moderate'] = ciCOH_normal, ciCOH_mild, ciCOH_moderate
ciCOH['chnInf_recchni'], ciCOH['chnInf_matchni'] = chnInf_recchni, chnInf_matchni
ciCOH['chnInf_brainarea'], ciCOH['chnInf_coord'] = chnInf_brainarea, chnInf_coord

In [29]:
try:
    import cPickle as pickle
except ImportError:  # python 3.x
    import pickle

with open(os.path.join(savefolder, savefilename), 'wb') as fp:
    pickle.dump(ciCOH, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
os.path.join(savefolder, savefilename)

'/Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/pipeline/NHP_Pinky/1_dataAnaly/FCAnaly/Rest/m1_calcciCOH_rest/ciCOH_rest_freq26_28.pickle'